# fonction de creation de tableau de taux d'echange


In [4]:
import requests
import pandas as pd
from io import BytesIO
import zipfile
import os

def download_and_process_csv(url, list_currency=None, start_date=None, end_date=None, folder_path=None):
    # Télécharger le fichier ZIP depuis l'URL
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content), 'r') as zip_ref:
        # Extraire le fichier CSV contenu dans le ZIP
        csv_filename = zip_ref.namelist()[0]
        extraction_path = 'dossier_taux_bce'
        zip_ref.extractall(extraction_path)
        with zip_ref.open(csv_filename) as csv_file:
            # Charger le CSV en DataFrame pandas
            df = pd.read_csv(csv_file)
    df.rename(columns={'Date': 'date'}, inplace=True)
    
    df['date'] = pd.to_datetime(df['date'])

    # Créer un nouveau DataFrame avec les colonnes de list_currency
    if list_currency:
        currency_columns = ['date'] + list_currency
        df = df[currency_columns]
    
    # Filtrer le DataFrame en fonction de start_date et end_date
    if start_date:
        if end_date:
            mask = (df['date'] >= start_date) & (df['date'] <= end_date)
            df = df.loc[mask]
        
        else:
            mask = df['date'] >= start_date
            df = df.loc[mask]

    else:
        if end_date: 
            mask = df['date'] <= end_date
            df = df.loc[mask]
   

    melted_df = df.melt(id_vars=['date'], var_name='key_change', value_name='taux').sort_values([ 'date','key_change'],ascending=False)
    melted_df['x_vers_euro'] = 1 / melted_df['taux']
    
    # Exporter le DataFrame en tant que CSV dans le dossier spécifié
    if folder_path:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        output_filename = os.path.join(folder_path, 'filtered_currency_data_abc.csv')
        melted_df.to_csv(output_filename, index=False)
        print(f"Fichier CSV exporté avec succès vers {output_filename}")
    else:
        folder_path=extraction_path
        output_filename = os.path.join(folder_path, 'filtered_currency_data_abc.csv')
        melted_df.to_csv(output_filename, index=False)
        print(f"Fichier CSV exporté avec succès vers {output_filename}")




In [6]:
# Exemple d'utilisation de la fonction
url ='https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.zip?bd049ef55462a7bf91e6643cd0735a01'
list_currency = ['USD','JPY','CNY','GBP']
# #list_currency = None
# start_date = None
# end_date = None
# folder_path = None
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-06-30')

#download_and_process_csv(url, list_currency, start_date, end_date, folder_path)
download_and_process_csv(url)

Fichier CSV exporté avec succès vers dossier_taux_bce/filtered_currency_data_abc.csv


In [18]:
import pandas as pd

In [30]:
filelink = 'dossier_taux_bce/eurofxref-hist.csv'

In [34]:
def recuperation_local(filelink):
    df = pd.read_csv(filelink)
    df.drop("Unnamed: 42",axis=1,inplace=True)
    df.rename(columns={'Date': 'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    melted_df = df.melt(id_vars=['date'], var_name='key_change', value_name='taux').sort_values([ 'date','key_change'],ascending=False)
    melted_df['x_vers_euro'] = 1 / melted_df['taux']
    melted_df['date'] = melted_df['date'].dt.strftime('%d/%m/%Y')
    #output_filename = os.path.join('filtered_currency_data.csv')
    melted_df.to_csv('local_currency_data.csv', index=False)



In [35]:
recuperation_local(filelink)

In [19]:
df = pd.read_csv("dossier_taux_bce/eurofxref-hist.csv")

In [20]:
df.columns

Index(['Date', 'USD', 'JPY', 'BGN', 'CYP', 'CZK', 'DKK', 'EEK', 'GBP', 'HUF',
       'LTL', 'LVL', 'MTL', 'PLN', 'ROL', 'RON', 'SEK', 'SIT', 'SKK', 'CHF',
       'ISK', 'NOK', 'HRK', 'RUB', 'TRL', 'TRY', 'AUD', 'BRL', 'CAD', 'CNY',
       'HKD', 'IDR', 'ILS', 'INR', 'KRW', 'MXN', 'MYR', 'NZD', 'PHP', 'SGD',
       'THB', 'ZAR', 'Unnamed: 42'],
      dtype='object')

In [21]:
df.drop("Unnamed: 42",axis=1,inplace=True)

In [22]:
df

,Date,USD,JPY,BGN,CYP,CZK,DKK,EEK,GBP,HUF,...,ILS,INR,KRW,MXN,MYR,NZD,PHP,SGD,THB,ZAR
0,2023-08-15,1.0926,159.04,1.9558,NaN,24.138,7.4519,NaN,0.85955,388.48,...,4.1156,91.0615,1460.70,18.7129,5.0650,1.8305,62.124,1.4829,38.667,20.9813
1,2023-08-14,1.0930,158.68,1.9558,NaN,24.038,7.4515,NaN,0.86215,382.78,...,4.0694,90.8180,1459.40,18.6209,5.0464,1.8305,62.132,1.4813,38.474,20.7881
2,2023-08-11,1.1004,158.97,1.9558,NaN,24.103,7.4510,NaN,0.86415,382.80,...,4.0994,91.0970,1459.31,18.7092,5.0475,1.8299,62.175,1.4850,38.547,20.7138
3,2023-08-10,1.1019,158.43,1.9558,NaN,24.223,7.4512,NaN,0.86313,385.13,...,4.0975,91.0983,1446.32,18.7458,5.0368,1.8123,61.820,1.4826,38.583,20.6870
4,2023-08-09,1.0968,157.35,1.9558,NaN,24.293,7.4511,NaN,0.86180,388.44,...,4.0753,90.8795,1441.47,18.8106,5.0146,1.8103,61.728,1.4761,38.361,20.8921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6301,1999-01-08,1.1659,130.09,NaN,0.58187,34.938,7.4433,15.6466,0.70940,250.15,...,NaN,NaN,1366.73,NaN,NaN,2.1557,NaN,1.9537,NaN,6.7855
6302,1999-01-07,1.1632,129.43,NaN,0.58187,34.886,7.4431,15.6466,0.70585,250.09,...,NaN,NaN,1337.16,NaN,NaN,2.1531,NaN,1.9436,NaN,6.8283
6303,1999-01-06,1.1743,131.42,NaN,0.58200,34.850,7.4452,15.6466,0.70760,250.67,...,NaN,NaN,1359.54,NaN,NaN,2.1890,NaN,1.9699,NaN,6.7307
6304,1999-01-05,1.1790,130.96,NaN,0.58230,34.917,7.4495,15.6466,0.71220,250.80,...,NaN,NaN,1373.01,NaN,NaN,2.2011,NaN,1.9655,NaN,6.7975


In [23]:
df.rename(columns={'Date': 'date'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])

melted_df = df.melt(id_vars=['date'], var_name='key_change', value_name='taux').sort_values([ 'date','key_change'],ascending=False)
melted_df['x_vers_euro'] = 1 / melted_df['taux']
melted_df['date'] = melted_df['date'].dt.strftime('%d/%m/%Y')

In [24]:
melted_df


,date,key_change,taux,x_vers_euro
252240,15/08/2023,ZAR,20.9813,0.047661
0,15/08/2023,USD,1.0926,0.915248
151344,15/08/2023,TRY,29.5648,0.033824
145038,15/08/2023,TRL,NaN,NaN
245934,15/08/2023,THB,38.6670,0.025862
...,...,...,...,...
119813,04/01/1999,CHF,1.6168,0.618506
176567,04/01/1999,CAD,1.8004,0.555432
170261,04/01/1999,BRL,NaN,NaN
18917,04/01/1999,BGN,NaN,NaN


In [26]:
melted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258546 entries, 252240 to 163955
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         258546 non-null  object 
 1   key_change   258546 non-null  object 
 2   taux         197315 non-null  float64
 3   x_vers_euro  197315 non-null  float64
dtypes: float64(2), object(2)
memory usage: 9.9+ MB
